<a href="https://colab.research.google.com/github/LivingstonTardzenyuy/Deep-Learning-with-TensorFlow/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Introduction to NLP Fundamentals in TensorFlow.

 NLP has the goal of deriving information out of natural language (could be sequence of text or words).
 Another common term for NLP problems is sequence to sequence problems (seq2seq).

In [2]:
# Check for GPU.

!nvidia-smi

Sat Mar  8 04:45:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
## Get helper functions.
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-03-08 04:45:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-03-08 04:45:08 (106 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get our text dataset.

The dataset we're going to be using is Kaggle's introduction to NLP dataset(text samples of Tweets labelled as diaster or not diaster).


In [4]:
!wget  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data.
unzip_data("nlp_getting_started.zip")

--2025-03-08 04:45:12--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.207, 74.125.130.207, 74.125.68.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   719KB/s    in 0.8s    

2025-03-08 04:45:13 (719 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Reading the file with pandas.
import pandas as pd

train_df  = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Lets' shuffle our dataFrame.
train_df_shuffled = train_df.sample(
    frac=1,   # Frac is what percentage of the data to shuffle, 1 is equal to 100
    random_state=42
)

In [8]:
# What are the totoal number of samples.

len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random sample examples.
import random
random_index = random.randint(0, len(train_df)-5)    # create random indexes not higher than the totla number of samples.
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
@chaosmagician97 awesome!! I saw he was bleeding pretty bad

---

Target: 1 (real disaster)
Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/4sf0rgn8Wo

---

Target: 0 (not real disaster)
Text:
Best movie you've ever seen? - Armageddon  http://t.co/qoUXIgdtbZ

---

Target: 0 (not real disaster)
Text:
Wholesale #WE Gon Rep That $hit At All Costs- Hazardous #WholeTeam3 #WholesaleEnt https://t.co/JWnXH9Q5ov

---

Target: 0 (not real disaster)
Text:
RT @WIRED: Reddit will now quarantine offensive content http://t.co/zlAGv1U5ZA

---



## Split data into training and validation sets.

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use trian_test_split to split training data into traning and validation sets.

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42
)

In [13]:
# Check the first 10 samples.
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text to numbers.

When dealing with text problems one of the first thing we have to do before we build the model is to convert the text to numbers. There are few ways to do this.

1. Tokenization: Direct mapping of Tokenization
2. Embedding: Create a matrix of feature vector for each token.

## Text Vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D


# Use the default TextVectorization parameters.
text_vectorizer = TextVectorization(
    max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
    standardize="lower_and_strip_punctuation", # how to process text.
    split="whitespace", # how to split tokens
    ngrams=None, # create groups of n-words?
    output_mode="int", # how to map tokens to numbers
    output_sequence_length=None, # how long should the output sequence of tokens be?
    # pad_to_max_tokens=True
)

In [18]:
# Find the average numbers of token (words) in the traning data.
average_number_of_token = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
average_number_of_token

15

In [19]:
# Setup text vectorization variables.
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = average_number_of_token # max length our sequences will be (e.g. how many words from a Tweet does our model see?)


# recreating the intance of text_vectorization.
text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [20]:
# Fit the text vectorizer to the traning text.

text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it.
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# Choose a random sentence from the training dataset and tokenize it.
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 My Instagram just blew up apparently I was featured on I am jazz tonight. How cool is that love her        

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  13, 2927,   29,  555,   27, 2193,    8,   23, 5659,   11,    8,
         160, 3722,  383,   62]])>

In [26]:
# Get the unique words in the vocabulary.
words_in_vocab = text_vectorizer.get_vocabulary()  # get all of the unique words.
top_5_words = words_in_vocab[:5]  # get the most commmon words.
bottom_5_words = words_in_vocab[-5:]  # get the least common words.
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating an Embedding using an Embedding layer.

To make our embedding, we're going to use Tensorflows embedding layer.

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The paramters we care most about for our embedding layer:
1. 'input_dim' = Size of our input vocab.
2. 'output_dim' = Size of the output embedding vector.

example, a value of 100 would mean each token gets represented by a vector 100 long.
3. 'input_length' = length of the sequences being passed to the embedding layer.

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,     # Neural networks works with numbers that are divible by 8.
)

embedding

<Embedding name=embedding_1, built=False>

In [29]:
# Get a random sentence from the training set.
random_sentence = random.choice(train_sentences)

print(f"Original text: \n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(
    text_vectorizer([random_sentence])
)

sample_embed


Original text: 
 @fewmoretweets all lives matter. Just not a fan of burning down buildings and stealing from your neighbors to 'protest'        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02070825,  0.00765298,  0.0105149 , ...,  0.04538855,
          0.02897546,  0.0303595 ],
        [-0.00820995,  0.0235048 ,  0.03008392, ..., -0.02235333,
         -0.04455527, -0.01259768],
        [ 0.0355008 ,  0.0271093 , -0.02990193, ...,  0.00348337,
          0.00912484, -0.01921435],
        ...,
        [ 0.04809816,  0.01661854,  0.03140613, ...,  0.01093914,
          0.04141638,  0.01721371],
        [-0.01980324,  0.00225959, -0.04708299, ..., -0.04418474,
         -0.02382002, -0.0047521 ],
        [ 0.00936316,  0.03075707,  0.0013768 , ..., -0.04521279,
         -0.03954964,  0.02100245]]], dtype=float32)>

In [30]:
# Check out a single tokne's embedding.

sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02070825,  0.00765298,  0.0105149 , -0.00507504,  0.02318213,
        -0.04947042, -0.04086571, -0.03938166, -0.01144361, -0.01977422,
         0.02117089, -0.03671043, -0.03658701, -0.01515947,  0.01380252,
        -0.02297257, -0.02620621,  0.00103694,  0.02324638, -0.00149842,
        -0.00116354,  0.00053809,  0.00918115,  0.00909287, -0.0295159 ,
         0.00633404, -0.00733791, -0.03546342, -0.03824443, -0.02414528,
        -0.01838853,  0.02430635,  0.04893638,  0.02299777, -0.04936258,
         0.01442727,  0.04046062, -0.02774061,  0.03003857, -0.02359157,
         0.01152213, -0.03856277,  0.00358927,  0.01859475, -0.0242699 ,
        -0.01922495,  0.02694965,  0.01821828, -0.03144068,  0.01169636,
        -0.04336661,  0.03838648,  0.00892727,  0.03824646, -0.02710755,
         0.02134088,  0.01113274, -0.02750068, -0.01745981,  0.03685081,
        -0.04983065,  0.0079801 , -0.00308383,  0.0137609 ,  0.00574913,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

we'll start with a baseline and move on from there.

* Model 0: Build a Naive Bayes (baseline): https://scikit-learn.org/stable/modules/naive_bayes.html

* Model 1: Feed-forward neural network (dense model)

* Model 2: LSTM model (RNN)

* Model 3: GRU model (RNN)

* Model 4: Bidirectional_LSTM model (RNN)

* Model 5: 1D Convolutional Neural Network(CNN)

* Model 6: TensorFlow Hub Pretrained Feature extractor (using transfer learning for NLP)

* Model 7: Same as model 6 with 10% of traning data


Wei'll use the standard stpes in modelling.

* Create a model.
* Build a model
* Fit the model
* Evaluate our model.
